## 📌 Mount Google Drive ke Colab

Untuk mengakses file yang tersimpan di Google Drive melalui Google Colab, kita perlu **mount (mengaitkan)** Google Drive ke lingkungan runtime Colab. Ini memungkinkan kita membaca atau menulis file secara langsung ke/dari Google Drive.

```python
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 📚 Import Library untuk Proyek Machine Learning Klasifikasi

- **pandas** (`pd`): Manipulasi dan analisis data dalam bentuk DataFrame.  
- **matplotlib.pyplot** (`plt`) & **seaborn** (`sns`): Visualisasi data (grafik dan plot).  
- **numpy** (`np`): Operasi numerik dan array.  
- **tensorflow** (`tf`): Framework utama untuk membangun dan melatih model deep learning.  
- **scikit-learn**:  
  - `train_test_split`: Membagi dataset menjadi data latih dan uji.  
  - `classification_report`, `confusion_matrix`: Evaluasi performa klasifikasi.  
  - `LabelEncoder`, `StandardScaler`: Preprocessing data (encoding label dan standarisasi fitur).  
- **tensorflow.keras.utils.to_categorical**: Mengubah label numerik menjadi format one-hot encoding.  
- **tensorflow.keras.models.Sequential, load_model**: Membangun dan memuat model neural network secara bertahap.  
- **tensorflow.keras.layers.Dense, Dropout**: Lapisan fully connected dan dropout untuk neural network.

---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout

## 📂 Memuat dan Memproses Dataset

- **Memuat file CSV** dari Google Drive menggunakan `pandas.read_csv()`.
- Dataset awalnya memiliki data dalam satu kolom dengan nilai yang dipisah oleh `;`.
- Menggunakan `.str.split(';', expand=True)` untuk memisahkan data menjadi beberapa kolom.
- Menetapkan nama kolom sesuai isi data:  
  `'No_Induk', 'Name', 'DOB', 'Gender', 'Age', 'Weight', 'Height', 'Check_up_month', 'Check_up_year'`.

---

In [ ]:
# Load the uploaded dataset
file_path = '/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan_real.csv'
dataset = pd.read_csv(file_path)
dataset = dataset.iloc[:, 0].str.split(';', expand=True)
dataset.columns = [
    'No_Induk', 'Name', 'DOB', 'Gender', 'Age',
    'Weight', 'Height', 'Check_up_month', 'Check_up_year'
]

dataset

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Check_up_month,Check_up_year
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81,Februari,2022
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10,75,Februari,2022
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9,76,Februari,2022
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96,Februari,2022
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88,Februari,2022
...,...,...,...,...,...,...,...,...,...
826,46,Sena Humaira Aviaji,04/11/2024,Perempuan,17,8.2,75,Mei,2025
827,47,Nabila Putri Calista,29/05/2024,Perempuan,17,9.1,74.2,Mei,2025
828,48,Kanaya Zunaira Mafaza,18/06/2024,Perempuan,19,8.9,78.1,Mei,2025
829,49,Muh. Davindra Rizky Pratama,29/07/2024,Laki-laki,19,9,73.7,Mei,2025


## 🔄 Konversi Nama Bulan ke Format Tanggal

- Membuat dictionary `bulan_map` untuk memetakan nama bulan bahasa Indonesia ke format angka (01–12).
- Mengubah kolom `Check_up_month` dari nama bulan menjadi angka bulan menggunakan `.map()`.
- Menggabungkan `Check_up_year` dan `Month_num` menjadi kolom tanggal lengkap (`Checkup_date`) dengan tanggal default `1`.
- Mengonversi hasil gabungan tersebut menjadi tipe `datetime` dengan `pd.to_datetime()` untuk memudahkan analisis waktu.

---

In [ ]:
bulan_map = {
    "Januari": "01", "Februari": "02", "Maret": "03", "April": "04", "Mei": "05",
    "Juni": "06", "Juli": "07", "Agustus": "08", "September": "09",
    "Oktober": "10", "November": "11", "Desember": "12"
}

# Ubah nama bulan ke angka
dataset["Month_num"] = dataset["Check_up_month"].map(bulan_map)

# Buat kolom datetime (dengan tanggal default 1)
dataset["Checkup_date"] = pd.to_datetime(dataset["Check_up_year"].astype(str) + "-" + dataset["Month_num"] + "-01")

dataset

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Check_up_month,Check_up_year,Month_num,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81,Februari,2022,02,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10,75,Februari,2022,02,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9,76,Februari,2022,02,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96,Februari,2022,02,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88,Februari,2022,02,2022-02-01
...,...,...,...,...,...,...,...,...,...,...,...
826,46,Sena Humaira Aviaji,04/11/2024,Perempuan,17,8.2,75,Mei,2025,05,2025-05-01
827,47,Nabila Putri Calista,29/05/2024,Perempuan,17,9.1,74.2,Mei,2025,05,2025-05-01
828,48,Kanaya Zunaira Mafaza,18/06/2024,Perempuan,19,8.9,78.1,Mei,2025,05,2025-05-01
829,49,Muh. Davindra Rizky Pratama,29/07/2024,Laki-laki,19,9,73.7,Mei,2025,05,2025-05-01


## 🧹 Membersihkan Dataset dan Menyimpan Hasil

- Menghapus kolom sementara `Check_up_month`, `Check_up_year`, dan `Month_num` yang sudah tidak diperlukan lagi setelah pembuatan kolom `Checkup_date`.  
- Menyimpan dataset yang sudah bersih dan siap analisis ke Google Drive dalam format CSV tanpa menyertakan indeks (`index=False`).

---

In [ ]:
dataset.drop(columns=['Check_up_month', 'Check_up_year', 'Month_num'], inplace=True)
dataset.to_csv('/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan.csv', index=False)

## 📥 Memuat Data untuk Prediksi

- Membaca file CSV hasil preprocessing (`data_pemeriksaan.csv`) dari Google Drive menggunakan `pandas.read_csv()`.  
- Menampilkan 10 baris pertama data dengan `head(10)` untuk memastikan data berhasil dimuat dan melihat sekilas isi dataset.

---

In [ ]:
predict_data = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan.csv')
predict_data.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01
5,6,Azqila Mauza Andinitya,24/10/2018,Perempuan,39,21.1,91.0,2022-02-01
6,7,Putri Nadya Silvana,01/10/2019,Perempuan,30,11.9,90.0,2022-02-01
7,8,Dea Fiedella,29/08/2021,Perempuan,6,6.9,65.0,2022-02-01
8,9,Alby Yafie Naditik,06/04/2019,Laki-laki,33,12.5,93.0,2022-02-01
9,10,Angel Almaira,12/06/2019,Perempuan,32,10.7,79.0,2022-02-01


## ⚙️ Memuat Model Deep Learning dan Menampilkan Arsitektur

- Memuat model neural network yang sudah dilatih dari file `.h5` menggunakan `load_model()`.  
- File model disimpan di Google Drive dengan path yang diberikan.  
- Menampilkan ringkasan (summary) arsitektur model dengan `model.summary()`, termasuk jumlah layer, tipe layer, dan jumlah parameter yang bisa dilatih.

---

In [ ]:
# Load model dari file .h5
model = load_model('/content/drive/MyDrive/Capstone-ML/Capstone/model_stunting.h5')

# Cek ringkasan arsitektur
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 245 (984.00 B)

 Trainable params: 243 (972.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

## 📊 Melihat Sekilas Data Prediksi

- Menampilkan 5 baris pertama dari DataFrame `predict_data` dengan fungsi `.head()`.  
- Tujuannya untuk memeriksa dan memastikan data yang akan digunakan untuk prediksi sudah benar dan sesuai format.

---

In [ ]:
predict_data.head()

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01


## 🔄 Membuat Salinan Data untuk Prediksi

- Membuat salinan (copy) dari DataFrame `predict_data` ke variabel baru `predict`.  
- Tujuannya agar data asli tetap aman dan tidak berubah saat dilakukan manipulasi atau preprocessing pada data prediksi.  
- Menampilkan 5 baris pertama dari data salinan menggunakan `.head()`.

---

In [ ]:
predict = predict_data.copy()
predict.head()

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01


In [ ]:
label = LabelEncoder()
predict['Gender'] = label.fit_transform(predict['Gender'])
predict.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,0,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,0,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,0,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,0,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,0,26,10.4,88.0,2022-02-01
5,6,Azqila Mauza Andinitya,24/10/2018,1,39,21.1,91.0,2022-02-01
6,7,Putri Nadya Silvana,01/10/2019,1,30,11.9,90.0,2022-02-01
7,8,Dea Fiedella,29/08/2021,1,6,6.9,65.0,2022-02-01
8,9,Alby Yafie Naditik,06/04/2019,0,33,12.5,93.0,2022-02-01
9,10,Angel Almaira,12/06/2019,1,32,10.7,79.0,2022-02-01


## 🔤 Encoding Label Kategori

- Membuat objek `LabelEncoder` dari scikit-learn untuk mengubah data kategori menjadi angka.  
- Mengubah kolom `Gender` (yang berupa teks, misalnya "Male" dan "Female") menjadi nilai numerik (0, 1, dst) dengan `fit_transform()`.  
- Menampilkan 10 baris pertama hasil encoding dengan `.head(10)`.

---

In [ ]:
scaler = StandardScaler()
fitur = ['Gender','Age','Height','Weight']
predict[fitur] = scaler.fit_transform(predict[fitur])
predict.head()

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,-1.105407,0.299261,-0.291210,-0.298971,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,-1.105407,-0.750513,-0.377340,-0.719280,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,-1.105407,-1.137272,-0.664439,-0.649229,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,-1.105407,0.244010,0.340408,0.751802,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,-1.105407,-0.198000,-0.262501,0.191390,2022-02-01


## 🧮 Melakukan Prediksi dan Menyimpan Hasil

- Menyiapkan data fitur `X` dari kolom yang sudah ditentukan (`fitur`).  
- Melakukan prediksi menggunakan model deep learning dengan `model.predict(X)`, menghasilkan probabilitas tiap kelas.  
- Mengambil kelas dengan probabilitas tertinggi menggunakan `np.argmax()` sebagai hasil prediksi akhir.  
- Menambahkan kolom `prediction` ke dataframe `predict` yang berisi kelas hasil prediksi.  
- Menyalin hasil prediksi ke kolom baru `Stunting_Status` di dataframe asli `predict_data`.  
- Menampilkan 10 sampel data secara acak menggunakan `.sample(10)` untuk melihat hasil prediksi.

---

In [ ]:
X = predict[fitur].values
prediksi = model.predict(X)

predicted_class = np.argmax(prediksi, axis=1)

predict['prediction'] = predicted_class
predict_data['Stunting_Status'] = predict['prediction'].copy()

predict_data.sample(10)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date,Stunting_Status
674,45,Abid Dilnawas Alfarizki Wicaksono,18/03/2024,Laki-laki,12,6.7,66.8,2024-11-01,1
613,34,Velisya Joanna Christanti,29/09/2021,Perempuan,35,11.8,91.4,2024-09-01,0
249,30,Aurora Aulia Arsy,03/10/2018,Perempuan,53,16.9,110.5,2023-05-01,0
172,14,Anisa Ramadani,06/06/2020,Perempuan,31,10.6,89.0,2022-12-01,0
112,21,Michelle Arsyila Fakhrana,26/10/2021,Perempuan,10,9.8,73.0,2022-08-01,0
189,9,Alby Yafie Naditik,06/04/2019,Laki-laki,44,13.9,100.0,2023-01-01,0
693,39,Elvano Daniswara Daiki,04/08/2021,Laki-laki,41,14.0,98.2,2024-12-01,0
419,39,Elvano Daniswara Daiki,04/08/2021,Laki-laki,30,11.8,91.0,2024-02-01,0
670,41,Arsyila Queenza Ayu Zea Elshanum,03/08/2023,Perempuan,18,10.3,82.4,2024-11-01,0
465,43,Sella Zevania,27/02/2024,Perempuan,5,5.3,63.0,2024-03-01,0


## 🏷️ Mengubah Label Numerik ke Kategori dan Menyimpan Data

- Membuat dictionary `label_map` untuk mengonversi kode numerik prediksi ke label kategori yang lebih mudah dipahami:  
  - `0` → "Normal"  
  - `1` → "Stunting"  
  - `2` → "Severely Stunting"
- Mengubah kolom `Stunting_Status` di `predict_data` dari angka ke label kategori menggunakan list comprehension.  
- Menyimpan dataframe hasil akhir ke file CSV baru di Google Drive tanpa menyertakan indeks (`index=False`).

---

In [ ]:
label_map = {
    0: 'Normal',
    1: 'Stunting',
    2: 'Severely Stunting'
}

predict_data['Stunting_Status'] = [label_map[i] for i in predict_data['Stunting_Status']]

# Simpan ke
predict_data.to_csv('/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan_with_status.csv', index=False)